## Process Boxscores

In [1]:
# vars <- list(currWk=3,
#              numWks=2
#             )

matchups <-  function(x){  #see line 140 for call
  rbind(data.frame(Week=x$Week,Team=x$Road,Oppn=gsub('@','',x$Home)),
        data.frame(Week=x$Week,Team=gsub('@','',x$Home),Oppn=x$Road))}

buildThisWk <- function(week,schedule){
  x <- schedule[which(schedule$Week==week),]
  
  # CREDIT: https://stackoverflow.com/questions/19297475/simplest-way-to-get-rbind-to-ignore-column-names
  x <- rbind(x[c(3,4)],setNames(rev(x[c(3,4)]),names(x[c(3,4)])))
  colnames(x) <- c('Team','Oppn')
  x <- cbind(Week=week,x)
  x
}

# wks 1 to vars$numWks
#aggPriorWks <- function(weeklybox, numWks=vars$numWks){ 
aggPriorWks <- function(weeklybox, numWks){ 
  aggregate(.~Team,
            data=subset(weeklybox,weeklybox$Week %in% c((max(weeklybox$Week)-vars$numWks+1):(max(weeklybox$Week))))[c(2,6:71)],
            FUN=sum)
}

getMetric <- function(weeklyBoxAgg, team, colnum){
  sapply(team,function(x){weeklyBoxAgg[which(weeklyBoxAgg$Team==x),][colnum]})
}

topSnapCounts <- function(tm, pos=c('QB','RB','TE','WR','CB','DE','DT','LB','S'), week, num=5){
  head(subset(snapcounts2018,snapcounts2018$team %in% tm & snapcounts2018$position %in% pos)[order(snapcounts2018[week+3],decreasing = TRUE),],num)
}

In [2]:
# 
# gameStats <- read.csv("D:/workbin/BigData/NFLScrape/nflBoxscoreScrape2018.csv",stringsAsFactors = FALSE)
# 
# leagueSched <- read.csv("D:/workbin/BigData/NFLScrape/2018_schedule_single.csv",stringsAsFactors = FALSE)
week=5
vars <- list(currWk=5,
             numWks=4
)

gameStats <- read.csv(sprintf("nflBoxscoreScrape2020_week%s.csv", week),stringsAsFactors = FALSE)
leagueSched <- read.csv(sprintf("2020_schedule_single_week%s.csv", week),stringsAsFactors = FALSE)

# snapcounts2018 <- data.frame(read.csv("nflSnapCount2018.csv",stringsAsFactors = FALSE))
# snapcounts2018[4:(length(snapcounts2018)-2)] <- as.numeric(snapcounts2018[4:length(snapcounts2018)])
# snapcounts2018 <- data.frame(read.csv("D:/workbin/BigData/NFLScrape/nflSnapCount2018.csv",stringsAsFactors = FALSE))
# 
# rm(snapcounts2018)
#Â

leagueSched$Date <- as.Date(leagueSched$Date,'%Y-%m-%d')

gameStats.orig <- gameStats

gameStats <- gameStats[which(gameStats$Gm!='Gm'),]

attach(gameStats)

gameStats$Date <- as.Date(as.character(Date),'%Y%m%d')
gameStats$Time.of.Pos <- strptime(Time.of.Pos,'%M:%S')$hour*60+strptime(Time.of.Pos,'%M:%S')$min
gameStats$X1st.Downs <- as.numeric(X1st.Downs)
gameStats$Rushing <- as.numeric(Rushing)
gameStats$Passing <- as.numeric(Passing)
gameStats$Penalty <- as.numeric(Penalty)
#Test below completion-attempt
gameStats$rdDwnCon <- as.numeric(regmatches(X3rd.Down.Conv,regexpr('\\d+',X3rd.Down.Conv)))
gameStats$rdDwnAtt <- abs(as.numeric(regmatches(X3rd.Down.Conv,regexpr('\\-\\d+',X3rd.Down.Conv))))
gameStats$thDwnCon <- as.numeric(regmatches(X4th.Down.Conv,regexpr('\\d+',X4th.Down.Conv)))
gameStats$thDwnAtt <- abs(as.numeric(regmatches(X4th.Down.Conv,regexpr('\\-\\d+',X4th.Down.Conv))))

gameStats$Total.Net.Yards <- as.numeric(Total.Net.Yards)
gameStats$Total.Plays <- as.numeric(Total.Plays)
gameStats$Avg.Gain<-NULL

gameStats$Net.Yards.Rushing <- as.numeric(Net.Yards.Rushing)
gameStats$Rush.Attempts <- as.numeric(Rush.Attempts)
gameStats$Avg.Rush.Yards<-NULL

gameStats$Net.Yards.Passing <- as.numeric(Net.Yards.Passing)
gameStats$PassCom <- as.numeric(regmatches(Comp....Att.,regexpr('\\d+',Comp....Att.)))
gameStats$PassAtt <- abs(as.numeric(regmatches(Comp....Att.,regexpr('\\-\\d+',Comp....Att.))))
gameStats$Comp....Att. <- NULL
gameStats$Yards.Per.Pass <- NULL

#gameStats$Sacked <- as.numeric(regmatches(Sacked...Yards.Lost,regexpr('\\d+',Sacked...Yards.Lost)))
#gameStats$SackYdsLost <- abs(as.numeric(regmatches(Sacked...Yards.Lost,regexpr('\\-\\d+',Sacked...Yards.Lost))))
#gameStats$Sacked...Yards.Lost <- NULL

gameStats$Penalties <- as.numeric(regmatches(Penalties...Yards,regexpr('\\d+',Penalties...Yards)))
gameStats$PenaltyYdsLost <- abs(as.numeric(regmatches(Penalties...Yards,regexpr('\\-\\d+',Penalties...Yards))))
gameStats$Penalties...Yards <- NULL

gameStats$Touchdowns <- as.numeric(Touchdowns)
gameStats$Rushing.TDs <- as.numeric(Rushing.TDs)
gameStats$Passing.TDs <- as.numeric(Passing.TDs)
gameStats$Other <- as.numeric(Other)

gameStats$Turnovers <- as.numeric(Turnovers)

gameStats$Fumbles <- as.numeric(regmatches(Fumbles...Lost,regexpr('\\d+',Fumbles...Lost)))
gameStats$FumblesLost <- abs(as.numeric(regmatches(Fumbles...Lost,regexpr('\\-\\d+',Fumbles...Lost))))
gameStats$Fumbles...Lost <- NULL

gameStats$Int..Thrown <- as.numeric(Int..Thrown)
gameStats$Punts...Avg <- NULL
gameStats$Return.Yards <- NULL
gameStats$PuntReturns <- as.numeric(regmatches(Punts...Returns,regexpr('\\d+',Punts...Returns)))
gameStats$PuntReturnYds <- abs(as.numeric(regmatches(Punts...Returns,regexpr('\\-\\d+',Punts...Returns))))
gameStats$KickReturns <- as.numeric(regmatches(Kickoffs...Returns,regexpr('\\d+',Kickoffs...Returns)))
gameStats$KickReturnYds <- abs(as.numeric(regmatches(Kickoffs...Returns,regexpr('\\-\\d+',Kickoffs...Returns))))
gameStats$Ints <- as.numeric(regmatches(Int....Returns,regexpr('\\d+',Int....Returns)))
gameStats$IntReturnYds <- abs(as.numeric(regmatches(Int....Returns,regexpr('\\-\\d+',Int....Returns))))

#gameStats$Kicking <- NULL
#gameStats$Kicking.XPMade <- as.numeric(regmatches(Extra.Points,regexpr('\\d+',Extra.Points)))
#gameStats$Kicking.XPAtt <- as.numeric(regmatches(Extra.Points,regexpr('\\d+$',Extra.Points)))
#gameStats$Kicking.FGMade <- as.numeric(regmatches(Field.Goals,regexpr('\\d+',Field.Goals)))
#gameStats$Kicking.FGAtt <- as.numeric(regmatches(Field.Goals,regexpr('\\d+$',Field.Goals)))

gameStats$RZScr <- as.numeric(regmatches(Red.Zone.Eff.,regexpr('\\d+',Red.Zone.Eff.)))
gameStats$RZAtt <- as.numeric(regmatches(Red.Zone.Eff.,regexpr('\\d+$',Red.Zone.Eff.)))

gameStats$GTGConv <- as.numeric(regmatches(Goal.to.Go.Eff.,regexpr('\\d+',Goal.to.Go.Eff.)))
gameStats$GTGAtt <- as.numeric(regmatches(Goal.to.Go.Eff.,regexpr('\\d+$',Goal.to.Go.Eff.)))

gameStats$Safeties <- NULL

colnames(gameStats)[4] <- 'TOP'
colnames(gameStats)[5] <- 'FirstDowns'
colnames(gameStats)[6] <- 'FrstDwnRsh'
colnames(gameStats)[7] <- 'FrstDwnPas'
colnames(gameStats)[8] <- 'FrstDwnPen'

colnames(gameStats)[11] <- 'TotalNetYds'
colnames(gameStats)[12] <- 'TotalPlays'

colnames(gameStats)[13] <- 'RushNetYds'
colnames(gameStats)[14] <- 'RushAtt'
# NEED AVG RUSH
colnames(gameStats)[15] <- 'PassNetYds'
# NEED AVG PASS
colnames(gameStats)[16] <- 'TDs'
colnames(gameStats)[17] <- 'TDsRush'
colnames(gameStats)[18] <- 'TDsPass'
colnames(gameStats)[19] <- 'TDsOthr'
colnames(gameStats)[21] <- 'IntLost'

gameStats$X3rd.Down.Conv <- NULL
gameStats$X4th.Down.Conv <- NULL
gameStats[20:26] <- NULL
# home: TRUE/FALSE, if team has @ in front
for (i in (1:nrow(gameStats))) gameStats$home[i] <- grepl(paste('@',gameStats$Team[i],sep=''),gameStats$Gm[i])

gameStats <- merge(gameStats,unique(leagueSched[c('Week','Date')]),by = 'Date',all=FALSE)

detach(gameStats)
sum(is.na(gameStats))

#dput(colnames(gameStats))
#https://stackoverflow.com/questions/5620885/how-does-one-reorder-columns-in-a-data-frame

gameStats <- gameStats[c("Week", "Gm", "Date", "home", "Team", "TOP",
                     "TotalPlays", "TotalNetYds",
                     "RushAtt", "RushNetYds",
                     "PassCom", "PassAtt", "PassNetYds",
                     "TDs", "TDsRush", "TDsPass", "TDsOthr",
                     "FirstDowns", "FrstDwnRsh", "FrstDwnPas","FrstDwnPen",
                     "thDwnAtt", "thDwnCon",
                     "GTGConv", "GTGAtt", "RZScr", "RZAtt",
                     "Ints", "IntReturnYds",
                     "Penalties", "PenaltyYdsLost", 
                     "IntLost", "Fumbles", "FumblesLost", "Turnovers",
                     "PuntReturns", "PuntReturnYds", "KickReturns", "KickReturnYds")]

#Inverse-Allowed
# High level approach: 1. Create field "Opponent" for each team for a given week (each row).
#                      2. Merge with itself on Team=Opponent

gameStats <- merge(gameStats,matchups(leagueSched),by=c('Week','Team'))
gameStats$Oppn <- as.character(gameStats$Oppn)

gameStats <- merge(gameStats,
                 data.frame(cbind(gameStats[c('Week','Team')],gameStats[c(6:27)],gameStats[c(29:31,33:39)])),
                 by.x=c('Week','Oppn'),by.y=c('Week','Team'))

gameStats$Oppn <- NULL
print(colnames(gameStats))


#Rename fields to "allowed"
colnames(gameStats) <- c('Week', 'Team', 'Gm', 'Date', 'home', 'TOP', 'TotalPlays', 
                       'TotalYds', 'RushAtt', 'RushYds', 'PassCom', 'PassAtt', 
                       'PassYds', 'TDs', 'TDsRush', 'TDsPass', 'TDsOthr', 
                       'FirstDowns', 'FrstDwnRsh', 'FrstDwnPas', 'FrstDwnPen', 
                       'thDwnAtt', 'thDwnCon', 'GTGConv', 
                       'GTGAtt', 'RZScr', 'RZAtt', 'Ints', 'IntReturnYds', 
                       'Penalties', 'PenaltyYdsLost', 'IntLost', 
                       'Fumbles', 'FumblesLost', 'Turnovers', 'PuntReturns', 
                       'PuntReturnYds', 'KickReturns', 'KickReturnYds', 'TOD', 
                       'PlaysDefended', 'NetYdsAllowed', 'RushPlaysDefended', 'RushYdsAllowed', 
                       'RecptnsAllowed', 'PassPlaysDefended', 'PassYdsAllowed', 'TDsAllowed', 'TDsRushAllowed', 
                       'TDsPassAllowed', 'TDsOthrAllowed', 'FirstDownsAllowed', 'FrstDwnRshAllowed', 'FrstDwnPasAllowed', 
                       'FrstDwnPenAllowed', 'thDwnAttDefended', 'thDwnConAllowed', 
                       'GTGConvAllowed', 'GTGAttDefended', 'RZScrAllowed', 'RZAttDefended', 'IntYdsLost', 
                       'PenaltiesForced', 'PenaltyYdsGained', 
                       'FumblesForced', 'FumblesRecovered', 'TurnoversForced', 'Punts', 
                       'PuntReturnYdsAllowed', 'KickOffs', 'KickReturnYdsAllowed')

[1] 0

 [1] "Week"             "Team"             "Gm"               "Date"            
 [5] "home"             "TOP.x"            "TotalPlays.x"     "TotalNetYds.x"   
 [9] "RushAtt.x"        "RushNetYds.x"     "PassCom.x"        "PassAtt.x"       
[13] "PassNetYds.x"     "TDs.x"            "TDsRush.x"        "TDsPass.x"       
[17] "TDsOthr.x"        "FirstDowns.x"     "FrstDwnRsh.x"     "FrstDwnPas.x"    
[21] "FrstDwnPen.x"     "thDwnAtt.x"       "thDwnCon.x"       "GTGConv.x"       
[25] "GTGAtt.x"         "RZScr.x"          "RZAtt.x"          "Ints"            
[29] "IntReturnYds.x"   "Penalties.x"      "PenaltyYdsLost.x" "IntLost"         
[33] "Fumbles.x"        "FumblesLost.x"    "Turnovers.x"      "PuntReturns.x"   
[37] "PuntReturnYds.x"  "KickReturns.x"    "KickReturnYds.x"  "TOP.y"           
[41] "TotalPlays.y"     "TotalNetYds.y"    "RushAtt.y"        "RushNetYds.y"    
[45] "PassCom.y"        "PassAtt.y"        "PassNetYds.y"     "TDs.y"           
[49] "TDsRush.y"        "TDs

In [3]:
head(gameStats)

Week,Team,Gm,Date,home,TOP,TotalPlays,TotalYds,RushAtt,RushYds,...,IntYdsLost,PenaltiesForced,PenaltyYdsGained,FumblesForced,FumblesRecovered,TurnoversForced,Punts,PuntReturnYdsAllowed,KickOffs,KickReturnYdsAllowed
5,NYJ,20201011_ARI@NYJ,2020-10-11,TRUE,27,64,285,28,123,...,0,10,59,1,0,1,3,8,1,31
5,CAR,20201011_CAR@ATL,2020-10-11,FALSE,33,64,437,27,124,...,0,3,40,1,0,1,1,3,2,47
5,CIN,20201011_CIN@BAL,2020-10-11,FALSE,34,65,205,28,70,...,12,4,25,0,0,1,3,10,1,30
5,TEN,20201013_BUF@TEN,2020-10-13,TRUE,27,62,334,34,139,...,0,10,56,2,1,3,1,1,6,138
5,ATL,20201011_CAR@ATL,2020-10-11,TRUE,26,64,369,25,166,...,0,4,25,0,0,0,1,3,3,64
5,TB,20201008_TB@CHI,2020-10-08,FALSE,31,64,344,21,111,...,0,7,65,1,0,1,1,6,3,119


In [4]:
print(colnames(gameStats))

 [1] "Week"                 "Team"                 "Gm"                  
 [4] "Date"                 "home"                 "TOP"                 
 [7] "TotalPlays"           "TotalYds"             "RushAtt"             
[10] "RushYds"              "PassCom"              "PassAtt"             
[13] "PassYds"              "TDs"                  "TDsRush"             
[16] "TDsPass"              "TDsOthr"              "FirstDowns"          
[19] "FrstDwnRsh"           "FrstDwnPas"           "FrstDwnPen"          
[22] "thDwnAtt"             "thDwnCon"             "GTGConv"             
[25] "GTGAtt"               "RZScr"                "RZAtt"               
[28] "Ints"                 "IntReturnYds"         "Penalties"           
[31] "PenaltyYdsLost"       "IntLost"              "Fumbles"             
[34] "FumblesLost"          "Turnovers"            "PuntReturns"         
[37] "PuntReturnYds"        "KickReturns"          "KickReturnYds"       
[40] "TOD"                  "PlaysDefe

## Analyze Boxscore

In [5]:
priorWks <- aggPriorWks(gameStats,vars$numWks)

priorWks <- cbind(priorWks[1],
                  GP=sapply(priorWks$Team,function(x)
                    {nrow(gameStats[which(gameStats$Team==x &
                                        gameStats$Week %in% c((max(gameStats$Week)-vars$numWks+1):(max(gameStats$Week)))),])
                    }),
                  priorWks[2:length(priorWks)]
                  )

attach(priorWks)

priorWks$TDsPassPerGame <- TDsPass/GP
priorWks$YdsPerPass <- PassYds/PassAtt
priorWks$FrstDwnByPass <- FrstDwnPas/FirstDowns

priorWks$TDsRushPerGame <- TDsRush/GP
priorWks$YdsPerRush <- RushYds/RushAtt
priorWks$FrstDwnByRush <- FrstDwnRsh/FirstDowns

# Rush/Pass Ratio plays, 1st downs in last X games
# priorWks$FrstDwnRshPasRatio <- FrstDwnRsh/FrstDwnPas
# priorWks$FrstDwnPasProp <- FrstDwnPas/FirstDowns
# priorWks$TDsRushPassRatio <- TDsRush/TDsPass
# priorWks$TDsPassRatio <- TDsPass/vars$numWks

priorWks$TDsPassAllowedPerGame <- TDsPassAllowed/GP
priorWks$YdsAllowedPerPass <- PassYdsAllowed/PassPlaysDefended
priorWks$FrstDwnByPassAllowed <- FrstDwnPasAllowed/FirstDownsAllowed

priorWks$TDsRushAllowedPerGame <- TDsRushAllowed/GP
priorWks$YdsAllowedPerRush <- RushYdsAllowed/RushPlaysDefended
priorWks$FrstDwnByRushAllowed <- FrstDwnRshAllowed/FirstDownsAllowed

# Rush/Pass Ratio plays, 1st downs in last X games
# priorWks$FrstDwnAllowedRshPasRatio <- FrstDwnRshAllowed/FrstDwnPasAllowed
# priorWks$FrstDwnPasProp <- FrstDwnPas/FirstDowns
# priorWks$TDsAllowedRushPassRatio <- TDsRushAllowed/TDsPassAllowed
# priorWks$TDsPassRatio <- TDsPass/vars$numWks

# fumbles per play ran in last X games
priorWks$FumbLostPerPlay <- FumblesLost/TotalPlays
priorWks$FumbForcedPerPlay <- FumblesForced/PlaysDefended

#Ints and Sacks
priorWks$IntLostPerPass <- IntLost/PassAtt
priorWks$IntPerPassPlayDefended <- Ints/PassPlaysDefended
#priorWks$SackedPerPassAtt <- Sacked/PassAtt
#priorWks$SacksPerPassPlayDefended <- Sacks/PassPlaysDefended

#Kick & Punt Returns
priorWks$YdsPerReturn <- (KickReturnYds+PuntReturnYds)/(KickReturns+PuntReturns)
priorWks$YdsPerReturnAllowed <- (KickReturnYdsAllowed+PuntReturnYdsAllowed)/(KickOffs+Punts)

#FGs & XPs allowed per game
#priorWks$FGAttDefendedPerGame <- Kicking.FGDefended/GP
#priorWks$XPAttDefendedPerGame <- Kicking.XPDefended/GP

# Matchup category factor for matches (usually product of team's and opponent's against)
detach(priorWks)
# rm(thisWk)

thisWk <- buildThisWk(vars$currWk,leagueSched)
#build team and oppn metrics, cbind with thisWk after


tw <- data.frame(lapply(thisWk,function(x){gsub('@','',x)}))

colnames(thisWk)
colnames(tw)
#verify
sum(is.na(priorWks))
sum(is.na(thisWk))
sum(is.na(tw))
# Team P TDs per Gm
# Team Yds per Pass Att.
# Team P% 1st Dwn
# Team R TDs per Gm
# Team Yds per Rush
# Team R% 1st Dwn

thisWk$PassTD <- as.numeric(getMetric(priorWks,tw$Team,69))* 
                 as.numeric(getMetric(priorWks,tw$Oppn,75))
thisWk$YdsPerPass <- as.numeric(getMetric(priorWks,tw$Team,70))*
                     as.numeric(getMetric(priorWks,tw$Oppn,76))
thisWk$Pass1stDwn <- as.numeric(getMetric(priorWks,tw$Team,71))*
                     as.numeric(getMetric(priorWks,tw$Oppn,77))

thisWk$RushTD <- as.numeric(getMetric(priorWks,tw$Team,72))*
                 as.numeric(getMetric(priorWks,tw$Oppn,78))
thisWk$YdsPerRush <- as.numeric(getMetric(priorWks,tw$Team,73))*
                     as.numeric(getMetric(priorWks,tw$Oppn,79))
thisWk$Rush1stDwn <-as.numeric(getMetric(priorWks,tw$Team,74))*
                    as.numeric(getMetric(priorWks,tw$Oppn,80))

thisWk$RZPctDiff <- (as.numeric(getMetric(priorWks,tw$Team,23))/as.numeric(getMetric(priorWks,tw$Team,24)))-(1-
                    (as.numeric(getMetric(priorWks,tw$Oppn,57))/as.numeric(getMetric(priorWks,tw$Oppn,58)))
                    )

#thisWk$Sacks <- as.numeric(getMetric(priorWks,tw$Team,102))+
#                as.numeric(getMetric(priorWks,tw$Oppn,101))
thisWk$TODiff <- as.numeric(getMetric(priorWks,tw$Team,25))+as.numeric(getMetric(priorWks,tw$Team,63))-
                 as.numeric(getMetric(priorWks,tw$Oppn,29))+as.numeric(getMetric(priorWks,tw$Oppn,31))
thisWk$FFmbls <- as.numeric(getMetric(priorWks,tw$Team,82))+
                 as.numeric(getMetric(priorWks,tw$Oppn,81))
thisWk$Intpns <- as.numeric(getMetric(priorWks,tw$Team,84))+
                 as.numeric(getMetric(priorWks,tw$Oppn,83))

#returning
thisWk$KickPuntReturns <- as.numeric(getMetric(priorWks,tw$Team,85))*
                          as.numeric(getMetric(priorWks,tw$Oppn,86))
#kicking
#thisWk$FGXPPct <- (as.numeric(getMetric(priorWks,tw$Team,41))+as.numeric(getMetric(priorWks,tw$Team,43)))/
#                  (as.numeric(getMetric(priorWks,tw$Team,42))+as.numeric(getMetric(priorWks,tw$Team,44)))
#Weak Pass D, Strong Rush D & RZ D
#thisWk$FGAttExpected <- as.numeric(getMetric(priorWks,tw$Oppn,105))
#thisWk$XPAttExpected <- as.numeric(getMetric(priorWks,tw$Oppn,106))

thisWkRank <- thisWk
for(i in 4:length(thisWk)){thisWkRank[i] <- rank(-thisWk[i],ties.method='average')}

[1] "Week" "Team" "Oppn"

[1] "Week" "Team" "Oppn"

[1] 0

[1] 0

[1] 0

In [6]:
head(thisWkRank)

,Week,Team,Oppn,PassTD,YdsPerPass,Pass1stDwn,RushTD,YdsPerRush,Rush1stDwn,RZPctDiff,TODiff,FFmbls,Intpns,KickPuntReturns
65,5,TB,@CHI,20.0,21,14.0,23.5,7,23.0,25.0,19,12,16.0,16
66,5,JAC,@HOU,12.0,20,5.0,23.5,21,22.0,15.0,19,11,2.5,1
67,5,CIN,@BAL,26.0,26,2.5,23.5,27,21.0,27.5,19,24,13.0,28
68,5,LAR,@WAS,12.0,4,18.5,6.5,22,8.5,16.0,19,6,23.5,27
69,5,LV,@KC,4.5,2,21.5,6.5,15,5.0,22.5,19,24,17.0,16
70,5,PHI,@PIT,12.0,14,7.0,6.5,5,25.5,18.0,6,10,23.5,19


In [19]:
colnames(priorWks)

[1] "Team"                   "GP"                     "TOP"                   
 [4] "TotalPlays"             "TotalYds"               "RushAtt"               
 [7] "RushYds"                "PassCom"                "PassAtt"               
[10] "PassYds"                "TDs"                    "TDsRush"               
[13] "TDsPass"                "TDsOthr"                "FirstDowns"            
[16] "FrstDwnRsh"             "FrstDwnPas"             "FrstDwnPen"            
[19] "thDwnAtt"               "thDwnCon"               "GTGConv"               
[22] "GTGAtt"                 "RZScr"                  "RZAtt"                 
[25] "Ints"                   "IntReturnYds"           "Penalties"             
[28] "PenaltyYdsLost"         "IntLost"                "Fumbles"               
[31] "FumblesLost"            "Turnovers"              "PuntReturns"           
[34] "PuntReturnYds"          "KickReturns"            "KickReturnYds"         
[37] "TOD"                    "PlaysDefended"          "NetYdsAllowed"         
[40] "RushPlaysDefended"      "RushYdsAllowed"         "RecptnsAllowed"        
[43] "PassPlaysDefended"      "PassYdsAllowed"         "TDsAllowed"            
[46] "TDsRushAllowed"         "TDsPassAllowed"         "TDsOthrAllowed"        
[49] "FirstDownsAllowed"      "FrstDwnRshAllowed"      "FrstDwnPasAllowed"     
[52] "FrstDwnPenAllowed"      "thDwnAttDefended"       "thDwnConAllowed"       
[55] "GTGConvAllowed"         "GTGAttDefended"         "RZScrAllowed"          
[58] "RZAttDefended"          "IntYdsLost"             "PenaltiesForced"       
[61] "PenaltyYdsGained"       "FumblesForced"          "FumblesRecovered"      
[64] "TurnoversForced"        "Punts"                  "PuntReturnYdsAllowed"  
[67] "KickOffs"               "KickReturnYdsAllowed"   "TDsPassPerGame"        
[70] "YdsPerPass"             "FrstDwnByPass"          "TDsRushPerGame"        
[73] "YdsPerRush"             "FrstDwnByRush"          "TDsPassAllowedPerGame" 
[76] "YdsAllowedPerPass"      "FrstDwnByPassAllowed"   "TDsRushAllowedPerGame" 
[79] "YdsAllowedPerRush"      "FrstDwnByRushAllowed"   "FumbLostPerPlay"       
[82] "FumbForcedPerPlay"      "IntLostPerPass"         "IntPerPassPlayDefended"
[85] "YdsPerReturn"           "YdsPerReturnAllowed"

In [7]:
head(thisWk)

,Week,Team,Oppn,PassTD,YdsPerPass,Pass1stDwn,RushTD,YdsPerRush,Rush1stDwn,RZPctDiff,TODiff,FFmbls,Intpns,KickPuntReturns
65,5,TB,@CHI,1,32.29566,0.3734568,0,27.93878,0.04938272,-0.3333333,0,0.01694915,0.04761905,225.00000
66,5,JAC,@HOU,4,34.78592,0.5289256,0,14.06250,0.05165289,3.0000000,0,0.01724138,0.11428571,1600.00000
67,5,CIN,@BAL,0,20.25000,0.5625000,0,6.25000,0.06250000,-1.0000000,0,0.00000000,0.05405405,16.00000
68,5,LAR,@WAS,4,100.00000,0.3025000,4,13.58449,0.12250000,2.0000000,0,0.03846154,0.00000000,19.61224
69,5,LV,@KC,9,124.57440,0.2975207,4,16.92735,0.13223140,0.0000000,0,0.00000000,0.04651163,225.00000
70,5,PHI,@PIT,4,45.18827,0.4900000,4,34.51562,0.04000000,1.0000000,2,0.02985075,0.00000000,150.06250


In [9]:
head(tw)

Week,Team,Oppn
5,TB,CHI
5,JAC,HOU
5,CIN,BAL
5,LAR,WAS
5,LV,KC
5,PHI,PIT


In [8]:
head(priorWks)

,Team,GP,TOP,TotalPlays,TotalYds,RushAtt,RushYds,PassCom,PassAtt,PassYds,...,FrstDwnByPassAllowed,TDsRushAllowedPerGame,YdsAllowedPerRush,FrstDwnByRushAllowed,FumbLostPerPlay,FumbForcedPerPlay,IntLostPerPass,IntPerPassPlayDefended,YdsPerReturn,YdsPerReturnAllowed
ARI,ARI,1,31,68,496,30,127,27,37,369,...,0.3809524,0,4.392857,0.4285714,0.00000000,0.00000000,0.02702703,0.00000000,9.75000,10.40000
ATL,ATL,1,26,64,369,25,166,21,37,203,...,0.7500000,0,4.592593,0.2083333,0.00000000,0.00000000,0.02702703,0.00000000,16.66667,16.75000
BAL,BAL,1,25,62,332,24,161,19,37,171,...,0.7500000,0,2.500000,0.2500000,0.00000000,0.04615385,0.02702703,0.03333333,10.00000,4.00000
BUF,BUF,1,32,69,370,22,95,29,46,275,...,0.5600000,3,4.088235,0.3600000,0.01449275,0.00000000,0.04347826,0.00000000,19.85714,18.75000
CAR,CAR,1,33,64,437,27,124,27,37,313,...,0.5238095,1,6.640000,0.4761905,0.00000000,0.01562500,0.00000000,0.02702703,16.75000,16.66667
CHI,CHI,1,28,59,241,16,33,30,42,208,...,0.6111111,0,5.285714,0.2222222,0.00000000,0.03125000,0.02380952,0.00000000,31.25000,15.00000


In [4]:
thisWkRankAvg <- as.data.frame(cbind(Week=thisWk$Week
                              ,Team=thisWk$Team
                              ,Oppn=thisWk$Oppn
                              ,QBWRTE=round(rowMeans(thisWkRank[,c(4:6,10)]),2)
                              ,RB=round(rowMeans(thisWkRank[,c(7:10)]),2)
                              ,DEF=round(rowMeans(thisWkRank[,c(11:13)]),2)
                              ,DLLB=round(rowMeans(thisWkRank[,c(11:13)]),2)
                              ,DB=round(rowMeans(thisWkRank[,c(11:13)]),2)
                              ),stringsAsFactors = FALSE)
thisWkRankAvg[4:8] <- lapply(thisWkRankAvg[4:8],FUN=as.numeric)

thisWkRankAvg[order(thisWkRankAvg$QBWRTE),]

,Week,Team,Oppn,QBWRTE,RB,DEF,DLLB,DB
661,5,@HOU,JAC,7.25,13.75,9.00,9.00,9.00
691,5,@KC,LV,8.00,14.12,17.33,17.33,17.33
781,5,@TEN,BUF,8.38,6.12,5.67,5.67,5.67
73,5,MIA,@SF,10.25,17.62,3.83,3.83,3.83
77,5,LAC,@NO,10.50,21.12,18.67,18.67,18.67
761,5,@SEA,MIN,10.88,8.62,9.33,9.33,9.33
741,5,@CLE,IND,11.25,17.12,14.67,14.67,14.67
771,5,@NO,LAC,11.25,12.62,19.83,19.83,19.83
71,5,CAR,@ATL,11.38,20.38,15.33,15.33,15.33
701,5,@PIT,PHI,12.12,10.12,15.67,15.67,15.67


In [5]:
thisWkRankAvg[order(thisWkRankAvg$RB),]

,Week,Team,Oppn,QBWRTE,RB,DEF,DLLB,DB
781,5,@TEN,BUF,8.38,6.12,5.67,5.67,5.67
711,5,@ATL,CAR,19.50,6.38,22.17,22.17,22.17
731,5,@SF,MIA,21.12,7.75,18.50,18.50,18.50
761,5,@SEA,MIN,10.88,8.62,9.33,9.33,9.33
751,5,@DAL,NYG,13.25,8.75,12.50,12.50,12.50
701,5,@PIT,PHI,12.12,10.12,15.67,15.67,15.67
721,5,@NYJ,ARI,19.50,10.38,16.00,16.00,16.00
72,5,ARI,@NYJ,15.12,11.62,22.17,22.17,22.17
76,5,MIN,@SEA,17.62,12.00,17.67,17.67,17.67
69,5,LV,@KC,12.62,12.25,20.00,20.00,20.00


In [6]:
thisWkRankAvg[order(thisWkRankAvg$DEF),]

,Week,Team,Oppn,QBWRTE,RB,DEF,DLLB,DB
671,5,@BAL,CIN,20.12,15.25,3.33,3.33,3.33
73,5,MIA,@SF,10.25,17.62,3.83,3.83,3.83
781,5,@TEN,BUF,8.38,6.12,5.67,5.67,5.67
75,5,NYG,@DAL,19.88,14.00,8.67,8.67,8.67
661,5,@HOU,JAC,7.25,13.75,9.00,9.00,9.00
761,5,@SEA,MIN,10.88,8.62,9.33,9.33,9.33
66,5,JAC,@HOU,13.00,20.38,10.83,10.83,10.83
651,5,@CHI,TB,14.25,20.88,11.17,11.17,11.17
751,5,@DAL,NYG,13.25,8.75,12.50,12.50,12.50
70,5,PHI,@PIT,12.75,13.75,13.17,13.17,13.17


In [7]:
thisWkRankAvg[order(thisWkRankAvg$DLLB),]

,Week,Team,Oppn,QBWRTE,RB,DEF,DLLB,DB
671,5,@BAL,CIN,20.12,15.25,3.33,3.33,3.33
73,5,MIA,@SF,10.25,17.62,3.83,3.83,3.83
781,5,@TEN,BUF,8.38,6.12,5.67,5.67,5.67
75,5,NYG,@DAL,19.88,14.00,8.67,8.67,8.67
661,5,@HOU,JAC,7.25,13.75,9.00,9.00,9.00
761,5,@SEA,MIN,10.88,8.62,9.33,9.33,9.33
66,5,JAC,@HOU,13.00,20.38,10.83,10.83,10.83
651,5,@CHI,TB,14.25,20.88,11.17,11.17,11.17
751,5,@DAL,NYG,13.25,8.75,12.50,12.50,12.50
70,5,PHI,@PIT,12.75,13.75,13.17,13.17,13.17


In [8]:
thisWkRankAvg[order(thisWkRankAvg$DB),]

,Week,Team,Oppn,QBWRTE,RB,DEF,DLLB,DB
671,5,@BAL,CIN,20.12,15.25,3.33,3.33,3.33
73,5,MIA,@SF,10.25,17.62,3.83,3.83,3.83
781,5,@TEN,BUF,8.38,6.12,5.67,5.67,5.67
75,5,NYG,@DAL,19.88,14.00,8.67,8.67,8.67
661,5,@HOU,JAC,7.25,13.75,9.00,9.00,9.00
761,5,@SEA,MIN,10.88,8.62,9.33,9.33,9.33
66,5,JAC,@HOU,13.00,20.38,10.83,10.83,10.83
651,5,@CHI,TB,14.25,20.88,11.17,11.17,11.17
751,5,@DAL,NYG,13.25,8.75,12.50,12.50,12.50
70,5,PHI,@PIT,12.75,13.75,13.17,13.17,13.17
